# Setup

In [5]:
import numpy as np
import pandas as pd
import os

In [74]:
jp = os.path.join
parent_dir = "C:\\Users\\chris\\Desktop\\cQuant Exercise\\"
data_dir = jp(parent_dir, "Data")
output_dir = jp(parent_dir, "Outputs")

# Task 1

In [15]:
fnames = os.listdir(data_dir)
data = pd.concat([pd.read_csv(jp(data_dir,f)) for f in fnames ])

# Task 2

In [83]:
print("Attributes: ", data.columns.values, "\nShape: ", data.shape)

Attributes:  ['Date' 'SettlementPoint' 'Price'] 
Shape:  (497320, 3)


In [192]:
data['Date'] = pd.to_datetime(data['Date'],infer_datetime_format = True)

In [35]:
#Set periods at each month-year
per = data.Date.dt.to_period("M")

In [45]:
MoY_Avg = data.groupby([per,'SettlementPoint']).mean()

In [41]:
#data.groupby([per,'SettlementPoint']).mean()

Price
Date    SettlementPoint           
2016-01 HB_BUSAVG        19.209301
        HB_HOUSTON       19.565726
        HB_HUBAVG        19.258696
        HB_NORTH         19.133306
        HB_SOUTH         19.210054
        HB_WEST          19.126559
        LZ_AEN           19.324664
        LZ_CPS           19.353844
        LZ_HOUSTON       19.563938
        LZ_LCRA          19.317460
        LZ_NORTH         19.187997
        LZ_RAYBN         19.447944
        LZ_SOUTH         19.481720
        LZ_WEST          19.519261
2016-02 HB_BUSAVG        15.288247
        HB_HOUSTON       15.464540
        HB_HUBAVG        15.269253
        HB_NORTH         15.292658
        HB_SOUTH         15.333592
        HB_WEST          14.988664
        LZ_AEN           15.448290
        LZ_CPS           15.520014
        LZ_HOUSTON       15.487989
        LZ_LCRA          15.488549
        LZ_NORTH         15.462213
        LZ_RAYBN         15.400431
        LZ_SOUTH         15.935862
        LZ_WEST          15.044483
2016-03 HB_BUSAVG        16.755626
        HB_HOUSTON       17.124953
...                            ...
2019-11 HB_BUSAVG        23.468375
        HB_HOUSTON       24.021500
        HB_HUBAVG        23.257236
        HB_NORTH         23.628611
        HB_PAN           11.661292
        HB_SOUTH         23.718556
        HB_WEST          21.660264
        LZ_AEN           24.408014
        LZ_CPS           25.575139
        LZ_HOUSTON       24.181611
        LZ_LCRA          24.485222
        LZ_NORTH         24.153611
        LZ_RAYBN         24.517569
        LZ_SOUTH         25.803708
        LZ_WEST          49.340847
2019-12 HB_BUSAVG        18.715081
        HB_HOUSTON       18.672500
        HB_HUBAVG        18.623374
        HB_NORTH         18.731989
        HB_PAN            8.817245
        HB_SOUTH         18.952513
        HB_WEST          18.136425
        LZ_AEN           19.051707
        LZ_CPS           19.997863
        LZ_HOUSTON       18.919852
        LZ_LCRA          19.456882
        LZ_NORTH         19.006801
        LZ_RAYBN         19.085067
        LZ_SOUTH         19.786882
        LZ_WEST          39.502594

[681 rows x 1 columns]

# Task 3

In [58]:
#Formatting Dates
MoY_Avg.reset_index(inplace=True)
MoY_Avg['Month']=MoY_Avg['Date'].apply(lambda x: str(x)[-2:])
MoY_Avg['Year'] = MoY_Avg['Date'].apply(lambda x: str(x)[:4])

In [64]:
#Formatting Table
MoY_Avg.drop("Date",1, inplace = True)
MoY_Avg = MoY_Avg[['SettlementPoint',"Year",'Month','Price']]
MoY_Avg.rename(columns = {'Price': "AveragePrice"}, inplace= True)

In [76]:
#Output
MoY_Avg.to_csv(jp(output_dir, "AveragePriceByMonth.csv"),index=False)

# Task 4

In [95]:
hubs = [ele for ele in data['SettlementPoint'].unique() if ele.startswith("HB_")]
print("Settlement Hubs: ",hubs)

Settlement Hubs:  ['HB_BUSAVG', 'HB_HOUSTON', 'HB_HUBAVG', 'HB_NORTH', 'HB_SOUTH', 'HB_WEST', 'HB_PAN']


In [136]:
data["Year"] = data['Date'].dt.year

In [114]:
target = data[data['SettlementPoint'].isin(hubs)]
target["Year"] = target['Date'].dt.year


C:\Users\chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [181]:
def hourly_price_volatility(df):
    output = []
    for hub in hubs:
        subset = df[df['SettlementPoint']==hub]
        for y in df['Year'].unique():
            subset2 = subset[subset['Year']==y]
            subset2['returns']=(subset2['Price']/subset2['Price'].shift(-1))

            pos_vals = subset2[subset2["returns"]>0]
            pos_vals['returns'] = np.log(pos_vals['returns'])
            pos_vals.replace([np.inf, -np.inf], np.nan,inplace=True)
            pos_vals.dropna(inplace=True)
            
            sd = np.std(pos_vals['returns'])
            hvol = sd*np.sqrt(len(pos_vals['returns']))
            
            output.append((hub,y,hvol))
            
    output = pd.DataFrame(output, columns = ["SettlementPoint","Year","HourlyVolatility"])
    
    
    return output
        

In [182]:
t5 = hourly_price_volatility(target)

C:\Users\chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [183]:
t5

,SettlementPoint,Year,HourlyVolatility
0,HB_BUSAVG,2016,17.408216
1,HB_BUSAVG,2017,14.419764
2,HB_BUSAVG,2018,18.739568
3,HB_BUSAVG,2019,23.336489
4,HB_HOUSTON,2016,17.907288
5,HB_HOUSTON,2017,15.953073
6,HB_HOUSTON,2018,18.738563
7,HB_HOUSTON,2019,23.202507
8,HB_HUBAVG,2016,17.612842
9,HB_HUBAVG,2017,14.849731


# Task 5

In [184]:
#Output
t5.to_csv(jp(output_dir,"HourlyVolatilityByYear.csv"),index=False)

# Task 6

In [190]:
#Shows the settlement hub with the highest overall volatility in each year
MaxVol = t5.groupby("Year").max()
MaxVol.to_csv(jp(output_dir,"MaxVolatilityByYear.csv"),index=True)

# Task 7

In [210]:
def formatting(df):
    for point in df['SettlementPoint'].unique():
        subset = df[df['SettlementPoint']==point]
        subset['hour'] = subset['Date'].dt.hour
        
        xform = subset.pivot_table(index=subset['Date'].dt.date, columns='hour', values='Price')
        xform.reset_index(inplace=True)
        xform.insert(0,'Variable', point)
        fname = "spot_" +point+".csv"
        xform.to_csv(jp(output_dir,"formattedSpotHistory\\",fname),index=False)
        print(point, " data formatted and saved.")
    return 'Formatting Complete'

In [211]:
formatting(data)

C:\Users\chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


HB_BUSAVG  data formatted and saved.
HB_HOUSTON  data formatted and saved.
HB_HUBAVG  data formatted and saved.
HB_NORTH  data formatted and saved.
HB_SOUTH  data formatted and saved.
HB_WEST  data formatted and saved.
LZ_AEN  data formatted and saved.
LZ_CPS  data formatted and saved.
LZ_HOUSTON  data formatted and saved.
LZ_LCRA  data formatted and saved.
LZ_NORTH  data formatted and saved.
LZ_RAYBN  data formatted and saved.
LZ_SOUTH  data formatted and saved.
LZ_WEST  data formatted and saved.
HB_PAN  data formatted and saved.


'Done'

# Mean Plots


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [212]:
hubs = [ele for ele in data['SettlementPoint'].unique() if ele.startswith("HB_")]
zones = [ele for ele in data['SettlementPoint'].unique() if ele.startswith("LZ_")]

To complete plots: take a subset of the monthly average prices dataframe for both hubs and zones
use matplotlib's plotting functions to build a multiple line plot, using a different color to plot each hub or zone
Set time (month-year) across the x-axis, Average price up the y-axis